# Symbolic Steady State Heat Equation

Here we use `MethodOfLines.jl` and `ModelingToolkit.jl` to solve the symbolic PDE system using the finite difference method (FDM).

- MethodOfLines.jl: http://methodoflines.sciml.ai/dev/
- ModelingToolkit.jl: https://mtk.sciml.ai/stable/

$$
\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} = 0
$$

In [ ]:
using ModelingToolkit
using MethodOfLines
using DomainSets
using NonlinearSolve
using Plots

In [ ]:
# Independent variables
@parameters x y
# Dependent variable
@variables u(..)

Dxx = Differential(x)^2
Dyy = Differential(y)^2

# PDE equation
eq = Dxx(u(x, y)) + Dyy(u(x, y)) ~ 0

# Boundary conditions
bcs = [u(0, y) ~ x * y,
       u(1, y) ~ x * y,
       u(x, 0) ~ x * y,
       u(x, 1) ~ x * y]

# Space (and time) domains
domains = [x ∈ Interval(0.0, 1.0),
           y ∈ Interval(0.0, 1.0)]

@named pdesys = PDESystem([eq], bcs, domains, [x, y], [u(x, y)])

In [ ]:
# Discretization of 2D sapce
N = 10
dx = 1 / N
dy = 1 / N

# Note that we pass in `nothing` for the time variable `t` here since we
# are creating a stationary problem without a dependence on time, only space.
discretization = MOLFiniteDifference([x => dx, y => dy], nothing, approx_order=2)

# Convert the PDE problem into an Nonlinear problem (steady-state problem)
prob = discretize(pdesys, discretization)
sol = NonlinearSolve.solve(prob, NewtonRaphson())

In [ ]:
# Get sptial resolutions and the 2D solution
discrete_x = sol[x]
discrete_t = sol[y]
u_sol = sol[u(x, y)]

heatmap(discrete_x, discrete_t, u_sol, 
        xlabel="x", ylabel="y", aspect_ratio=:equal,
        xlims=(0.0, 1.0), ylims=(0.0, 1.0), 
        title="Steady State Heat Equation")

## Runtime information

In [ ]:
versioninfo()

In [ ]:
using Pkg
Pkg.status()